In [0]:
%pip install Faker psycopg2-binary

In [0]:
import psycopg2
import json
import random
from faker import Faker
from datetime import datetime, timedelta
import os

host = dbutils.secrets.get(scope="project_secrets", key="supabase_host")
user = dbutils.secrets.get(scope="project_secrets", key="supabase_user")
password = dbutils.secrets.get(scope="project_secrets", key="supabase_password")
port = dbutils.secrets.get(scope="project_secrets", key="supabase_port")
p_day = dbutils.widgets.get("p_day")

SUPABASE_CONFIG = {
    "host": host,
    "database": "postgres",
    "user": user,
    "password": password,
    "port": port
}

VOLUME_PATH = "/Volumes/fleetops_project/default/landing_zone"
BATCH_DAY = 1  

fake = Faker('pt_BR')

def get_db_connection():
    return psycopg2.connect(**SUPABASE_CONFIG)

def run_day_1_genesis():
    conn = get_db_connection()
    cur = conn.cursor()
    
    print("Executing DAY 1: Cleaning and Populating")
    
    cur.execute("TRUNCATE TABLE driver_assignments CASCADE;")
    cur.execute("TRUNCATE TABLE trucks CASCADE;")
    cur.execute("TRUNCATE TABLE drivers CASCADE;")
    
    truck_ids = []
    for _ in range(10):
        cur.execute("""
            INSERT INTO trucks (plate, model, manufacture_year, capacity_kg)
            VALUES (%s, %s, %s, %s) RETURNING truck_id;
        """, (fake.license_plate(), random.choice(['Volvo FH', 'Scania R450', 'Mercedes Actros']), random.randint(2018, 2024), 30000))
        truck_ids.append(cur.fetchone()[0])
        
    driver_ids = []
    for _ in range(15):
        cur.execute("""
            INSERT INTO drivers (name, cnh, cnh_type, contract_date)
            VALUES (%s, %s, %s, %s) RETURNING driver_id;
        """, (fake.name(), fake.bothify(text='###########'), 'E', fake.date_between(start_date='-5y', end_date='today')))
        driver_ids.append(cur.fetchone()[0])
        
    active_assignments = [] 
    
    for t_id in truck_ids:
        d_id = driver_ids.pop(0)
        
        cur.execute("""
            INSERT INTO driver_assignments (driver_id, truck_id, start_date, is_active)
            VALUES (%s, %s, CURRENT_DATE - 2, TRUE);
        """, (d_id, t_id))
        
        cur.execute("SELECT plate FROM trucks WHERE truck_id = %s", (t_id,))
        plate = cur.fetchone()[0]
        
        active_assignments.append((d_id, t_id, plate))
        
    conn.commit()
    cur.close()
    conn.close()


def run_day_2_changes():
    conn = get_db_connection()
    cur = conn.cursor()
    
    print("Executing DAY 2: Driver Changes")
    
    cur.execute("""
        SELECT da.assignment_id, da.driver_id, da.truck_id, t.plate 
        FROM driver_assignments da
        JOIN trucks t ON da.truck_id = t.truck_id
        WHERE da.is_active = TRUE
    """)

    
    current_assignments = cur.fetchall() 
    
    active_assignments_for_telemetry = []
    
    target = current_assignments[0] 
    assign_id, old_driver_id, truck_id, plate = target
    
    cur.execute("""
            UPDATE driver_assignments 
            SET is_active = FALSE, 
                updated_at = NOW()
            WHERE assignment_id = %s;
        """, (assign_id,))
    

    cur.execute("SELECT driver_id FROM drivers WHERE driver_id != %s LIMIT 1", (old_driver_id,))
    new_driver_id = cur.fetchone()[0]
    
    cur.execute("""
        INSERT INTO driver_assignments (driver_id, truck_id, start_date, is_active)
        VALUES (%s, %s, CURRENT_DATE, TRUE);
    """, (new_driver_id, truck_id))
    
    print(f"CRITICAL CHANGE: Truck {plate} changed driver (ID {old_driver_id} -> {new_driver_id}) !!!")
    
    active_assignments_for_telemetry.append((new_driver_id, truck_id, plate))
    
    for item in current_assignments[1:]:
        active_assignments_for_telemetry.append((item[1], item[2], item[3]))
        
    conn.commit()
    cur.close()
    conn.close()
    
    print("DAY 2 Successfully Completed!")

if BATCH_DAY == 1:
    run_day_1_genesis()
elif BATCH_DAY == 2:
    run_day_2_changes()
else:
    print("Invalid day. Use 1 or 2.")